In [1]:
# 분석
import pandas as pd
import numpy as np

# 시각화
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
X_train = pd.read_csv('train_features.csv')
y_train = pd.read_csv('train_labels.csv')
X_test = pd.read_csv('test_features.csv')
submission = pd.read_csv('sample_submission.csv')
display(X_train.head(3))
display(y_train.head(3))
display(X_test.head(3))

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629


,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043


In [5]:
X_pivot_train = pd.pivot_table(data = X_train, # X_train의 데이터를 통해서
                               values = X_train.columns[2:],  # id와 time을 제외한 피쳐를 대상으로
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',         # 합, 평균
                                          'median','min','max', # 중앙값 최소값, 최대값
                                          'std','var'           # 베셀 보정 표본 표준편차, 비편향 편차 의 값을 구합니다.
                                         ]
                              )

X_pivot_test = pd.pivot_table(data = X_test, #
                               values = X_test.columns[2:], 
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',        
                                          'median','min','max',
                                          'std','var'          
                                         ]
                              )

display(X_pivot_train.head(3))
display(X_pivot_test.head(3))

sum                                                                 \
         acc_x       acc_y       acc_z         gy_x         gy_y         gy_z   
id                                                                              
0   558.797337 -131.082711 -222.252919 -1119.161589 -2015.703683   709.264425   
1  -459.948117 -190.354639   -2.534051  6642.960123  1044.284884   835.976169   
2    23.901616  -49.441742  375.607013 -5083.770868   358.725917  1831.974458   

        mean                                 ...       std             \
       acc_x     acc_y     acc_z       gy_x  ...     acc_z       gy_x   
id                                           ...                        
0   0.931329 -0.218471 -0.370422  -1.865269  ...  0.135131  13.284216   
1  -0.766580 -0.317258 -0.004223  11.071600  ...  0.499395  79.244561   
2   0.039836 -0.082403  0.626012  -8.472951  ...  0.248807  25.422926   

                                var                                   \
          gy_y       gy_z     acc_x     acc_y     acc_z         gy_x   
id                                                                     
0    24.300479  25.275185  0.036664  0.031375  0.018260   176.470384   
1    96.005289  75.545343  0.245548  0.113175  0.249396  6279.700472   
2   118.956646  13.920337  0.506904  0.021646  0.061905   646.325142   

                               
            gy_y         gy_z  
id                             
0     590.513292   638.834979  
1    9217.015511  5707.098884  
2   14150.683677   193.775778  

[3 rows x 42 columns]

sum                                                     \
           acc_x       acc_y       acc_z          gy_x         gy_y   
id                                                                    
3125 -611.238360  -11.744605 -139.355669  -1911.076959  1639.123438   
3126 -313.705824  367.296809  -42.655405 -10644.915365  4184.863263   
3127  304.167948  542.291164  -84.658968  -1307.846921 -1350.871152   

                       mean                                 ...       std  \
             gy_z     acc_x     acc_y     acc_z       gy_x  ...     acc_z   
id                                                          ...             
3125 -1200.410049 -1.018731 -0.019574 -0.232259  -3.185128  ...  0.174672   
3126 -2162.747150 -0.522843  0.612161 -0.071092 -17.741526  ...  0.383800   
3127  -235.904841  0.506947  0.903819 -0.141098  -2.179745  ...  0.152077   

                                            var                      \
           gy_x       gy_y       gy_z     acc_x     acc_y     acc_z   
id                                                                    
3125  12.897967  31.993022  12.251648  0.055806  0.008398  0.030510   
3126  51.625096  45.706311  61.604867  0.291264  0.110899  0.147302   
3127  22.770845  13.467885  23.041463  0.048371  0.036666  0.023128   

                                             
             gy_x         gy_y         gy_z  
id                                           
3125   166.357553  1023.553453   150.102867  
3126  2665.150566  2089.066820  3795.159662  
3127   518.511372   181.383940   530.909012  

[3 rows x 42 columns]

In [6]:
X_pivot_train.columns[:5]

MultiIndex([('sum', 'acc_x'),
            ('sum', 'acc_y'),
            ('sum', 'acc_z'),
            ('sum',  'gy_x'),
            ('sum',  'gy_y')],
           )

In [7]:
X_columns = [agg + '_' + column for agg,column in X_pivot_train.columns]
X_pivot_train.columns = X_columns
X_pivot_test.columns = X_columns
display(X_pivot_train.head())

,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,...,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
id,,,,,,,,,,,,,,,,,,,,,
0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,-1.865269,...,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,11.071600,...,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,-8.472951,...,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,17.744167,...,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,-4.819638,...,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


In [8]:
X_pivot_train = X_pivot_train.reset_index()
X_pivot_test = X_pivot_test.reset_index()

display(X_pivot_train.head())
display(X_pivot_test.head())

,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,...,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,...,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,...,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,...,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,...,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,...,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,...,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,3125,-611.238360,-11.744605,-139.355669,-1911.076959,1639.123438,-1200.410049,-1.018731,-0.019574,-0.232259,...,0.174672,12.897967,31.993022,12.251648,0.055806,0.008398,0.030510,166.357553,1023.553453,150.102867
1,3126,-313.705824,367.296809,-42.655405,-10644.915365,4184.863263,-2162.747150,-0.522843,0.612161,-0.071092,...,0.383800,51.625096,45.706311,61.604867,0.291264,0.110899,0.147302,2665.150566,2089.066820,3795.159662
2,3127,304.167948,542.291164,-84.658968,-1307.846921,-1350.871152,-235.904841,0.506947,0.903819,-0.141098,...,0.152077,22.770845,13.467885,23.041463,0.048371,0.036666,0.023128,518.511372,181.383940,530.909012
3,3128,-346.561617,-366.333946,18.891749,485.147442,-1790.981310,-14.590798,-0.577603,-0.610557,0.031486,...,0.326569,42.818157,45.069932,37.967372,0.186376,0.054569,0.106647,1833.394532,2031.298793,1441.521366
4,3129,-443.184021,109.521180,240.781103,-858.922755,865.419381,3447.298941,-0.738640,0.182535,0.401302,...,0.261848,92.301963,67.911174,43.353007,0.093512,0.098781,0.068565,8519.652350,4611.927587,1879.483194


In [9]:
print(X_pivot_train.shape)
print(X_pivot_test.shape)

(3125, 43)
(782, 43)


In [10]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [1]:
ftr = X_pivot_train
target = y_train['label']

NameError: name 'X_pivot_train' is not defined

In [12]:
X_train, X_val, y_train, y_val = train_test_split(ftr, target, test_size=0.2, random_state=156)

lgbm_wrapper = LGBMClassifier(n_estimators=1000, num_leaves=100, subsample=0.8,
                      min_child_samples=320, max_depth=30)

evals = [(X_val, y_val)]
lgbm_wrapper.fit(X_train, y_train, early_stopping_rounds=100, eval_metric='logloss', eval_set=evals, verbose=True)
preds = lgbm_wrapper.predict(X_pivot_test)
pred_proba = lgbm_wrapper.predict_proba(X_pivot_test)[:,:]

[1]	valid_0's multi_logloss: 1.85599
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 1.67307
[3]	valid_0's multi_logloss: 1.57024
[4]	valid_0's multi_logloss: 1.50163
[5]	valid_0's multi_logloss: 1.44535
[6]	valid_0's multi_logloss: 1.40027
[7]	valid_0's multi_logloss: 1.36106
[8]	valid_0's multi_logloss: 1.32294
[9]	valid_0's multi_logloss: 1.29436
[10]	valid_0's multi_logloss: 1.27216
[11]	valid_0's multi_logloss: 1.25699
[12]	valid_0's multi_logloss: 1.24278
[13]	valid_0's multi_logloss: 1.22688
[14]	valid_0's multi_logloss: 1.2128
[15]	valid_0's multi_logloss: 1.20063
[16]	valid_0's multi_logloss: 1.19083
[17]	valid_0's multi_logloss: 1.18263
[18]	valid_0's multi_logloss: 1.17727
[19]	valid_0's multi_logloss: 1.17005
[20]	valid_0's multi_logloss: 1.16778
[21]	valid_0's multi_logloss: 1.16705
[22]	valid_0's multi_logloss: 1.16466
[23]	valid_0's multi_logloss: 1.16005
[24]	valid_0's multi_logloss: 1.15838
[25]	valid_0's multi_logloss: 1.1603

In [13]:
pp =pd.DataFrame(pred_proba) 
pp

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.000074,0.000080,0.000089,0.000174,0.000113,0.000099,0.000094,0.000099,0.000357,0.000470,...,0.000513,0.000081,0.000052,0.000099,0.000136,0.000150,0.000071,0.002627,0.000094,0.000215
1,0.000487,0.000166,0.000185,0.000194,0.000284,0.000205,0.000195,0.000205,0.000739,0.000243,...,0.000176,0.000078,0.000107,0.000205,0.000282,0.000311,0.000146,0.000166,0.000195,0.000479
2,0.001329,0.002869,0.002336,0.003812,0.004648,0.002948,0.002459,0.002950,0.009312,0.003069,...,0.004586,0.000986,0.001356,0.006197,0.003558,0.005281,0.001847,0.002092,0.037099,0.083502
3,0.001086,0.000177,0.000190,0.000208,0.000250,0.000219,0.000209,0.000234,0.000791,0.000260,...,0.000188,0.000084,0.000115,0.000219,0.000302,0.000333,0.000157,0.000177,0.000209,0.000751
4,0.000130,0.000150,0.000167,0.000176,0.000211,0.000185,0.000176,0.000185,0.000668,0.000220,...,0.000159,0.000071,0.000097,0.000185,0.000255,0.000281,0.000132,0.000150,0.000176,0.000433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,0.000370,0.000229,0.000255,0.000268,0.005118,0.000292,0.000269,0.000282,0.001397,0.000335,...,0.000242,0.000108,0.000148,0.000282,0.000389,0.000429,0.000202,0.000229,0.000177,0.000659
778,0.000077,0.000121,0.000135,0.000142,0.000171,0.000149,0.000143,0.000150,0.000540,0.000178,...,0.000128,0.000057,0.000079,0.000150,0.000206,0.000227,0.000107,0.000121,0.000143,0.000350
779,0.000125,0.000136,0.000152,0.000159,0.000191,0.000168,0.000160,0.000168,0.000606,0.000199,...,0.000144,0.000064,0.000088,0.000168,0.000231,0.000255,0.000120,0.000136,0.000160,0.000393
780,0.000733,0.002131,0.001266,0.001327,0.001596,0.001399,0.000469,0.001397,0.008079,0.001658,...,0.001197,0.000534,0.000734,0.006480,0.001927,0.002128,0.001000,0.001133,0.002360,0.005684


In [14]:
pred_proba.shape

(782, 61)

In [15]:
submission.shape

(782, 62)

In [18]:
submission.iloc[:,1:] = pred_proba
submission

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000074,0.000080,0.000089,0.000174,0.000113,0.000099,0.000094,0.000099,0.000357,...,0.000513,0.000081,0.000052,0.000099,0.000136,0.000150,0.000071,0.002627,0.000094,0.000215
1,3126,0.000487,0.000166,0.000185,0.000194,0.000284,0.000205,0.000195,0.000205,0.000739,...,0.000176,0.000078,0.000107,0.000205,0.000282,0.000311,0.000146,0.000166,0.000195,0.000479
2,3127,0.001329,0.002869,0.002336,0.003812,0.004648,0.002948,0.002459,0.002950,0.009312,...,0.004586,0.000986,0.001356,0.006197,0.003558,0.005281,0.001847,0.002092,0.037099,0.083502
3,3128,0.001086,0.000177,0.000190,0.000208,0.000250,0.000219,0.000209,0.000234,0.000791,...,0.000188,0.000084,0.000115,0.000219,0.000302,0.000333,0.000157,0.000177,0.000209,0.000751
4,3129,0.000130,0.000150,0.000167,0.000176,0.000211,0.000185,0.000176,0.000185,0.000668,...,0.000159,0.000071,0.000097,0.000185,0.000255,0.000281,0.000132,0.000150,0.000176,0.000433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.000370,0.000229,0.000255,0.000268,0.005118,0.000292,0.000269,0.000282,0.001397,...,0.000242,0.000108,0.000148,0.000282,0.000389,0.000429,0.000202,0.000229,0.000177,0.000659
778,3903,0.000077,0.000121,0.000135,0.000142,0.000171,0.000149,0.000143,0.000150,0.000540,...,0.000128,0.000057,0.000079,0.000150,0.000206,0.000227,0.000107,0.000121,0.000143,0.000350
779,3904,0.000125,0.000136,0.000152,0.000159,0.000191,0.000168,0.000160,0.000168,0.000606,...,0.000144,0.000064,0.000088,0.000168,0.000231,0.000255,0.000120,0.000136,0.000160,0.000393
780,3905,0.000733,0.002131,0.001266,0.001327,0.001596,0.001399,0.000469,0.001397,0.008079,...,0.001197,0.000534,0.000734,0.006480,0.001927,0.002128,0.001000,0.001133,0.002360,0.005684
